# Dataset : 

- List of files to read
- File to save to (can be created or appended)
- Arbitrary label associated with the region, mask or whatsoever

In [125]:
import numpy as np
import pandas as pd
import os
import glob
import sys 

sys.path.append('C:/Users/nilso/Documents/EPFL/MA4/PDS Turberg/Rock_Cuttings_Characterisation/')

from Lamp.AttrDict.AttrDict import *

### Train

In [126]:
ifile = 'dataset_train.yaml'

In [127]:
if os.path.isfile(ifile) and os.path.splitext(ifile)[-1] in [".yaml",".yml"]:
    inputs = AttrDict.from_yaml_path(ifile) # change to argv
else:
    raise AssertionError("Wrong input type")

In [128]:
paths = []
labels = []

for key, items in inputs.Train.items():
    for folder in items.Folder:

        path = glob.glob(inputs.rootData + folder + '*.png')
        label = [items.Label] * len(path)

        paths.append(path)
        labels.append(label)

In [129]:
list_of_paths = [item for elem in paths for item in elem]
list_of_labels = [item for elem in labels for item in elem] 

In [130]:
df_train = pd.DataFrame(np.array([list_of_paths,list_of_labels]).T,columns=['Paths','Label'])

In [131]:
df_train['Paths'] = '../' + df_train['Paths'].str.split('/',expand=True,n=9)[9]

In [133]:
df_train_bbox = df_train.loc[df_train.Paths.str.find('bbox',start=-16) > -1, :].reset_index(drop=True)
df_train_mar = df_train.loc[df_train.Paths.str.find('mar',start=-16) > -1, :].reset_index(drop=True)

In [134]:
df_train_bbox.to_csv('train_bbox.csv')
df_train_mar.to_csv('train_mar.csv')

### Test

In [117]:
ifile = 'dataset_test.yaml'

In [118]:
if os.path.isfile(ifile) and os.path.splitext(ifile)[-1] in [".yaml",".yml"]:
    inputs = AttrDict.from_yaml_path(ifile) # change to argv
else:
    raise AssertionError("Wrong input type")

In [119]:
paths = []
labels = []

for key, items in inputs.Train.items():
    for folder in items.Folder:

        path = glob.glob(inputs.rootData + folder + '*.png')
        label = [items.Label] * len(path)

        paths.append(path)
        labels.append(label)

In [120]:
list_of_paths = [item for elem in paths for item in elem]
list_of_labels = [item for elem in labels for item in elem]

In [121]:
df_train = pd.DataFrame(np.array([list_of_paths,list_of_labels]).T,columns=['Paths','Label'])

In [122]:
df_train['Paths'] = '../' + df_train['Paths'].str.split('/',expand=True,n=9)[9]

In [123]:
df_train_bbox = df_train.loc[df_train.Paths.str.find('bbox',start=-16) > -1, :].reset_index(drop=True)
df_train_mar = df_train.loc[df_train.Paths.str.find('mar',start=-16) > -1, :].reset_index(drop=True)

In [124]:
df_train_bbox.to_csv('test_bbox.csv')
df_train_mar.to_csv('test_mar.csv')